In [1]:
import os
import json
import lyricwikia
import pickle
import sys

In [2]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from data.metrics import recall
from visualization.plot_utils import write_latex_table, summary_pooling_table, pooling_plots, pandas_settings, plot_settings,  save_result
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.tracks_info import TrackInfo
from collections import OrderedDict
from itertools import product
from data_utils import product_size
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [3]:
from features.tracks_info import TrackInfo
from features.popularity_feature import Popularity

tracks = TrackInfo(subset='5k-')
popularity = Popularity()

In [4]:
import json
from random import randint
import os
import json
import lyricwikia
import pickle
import sys
import numpy as np

def artists_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['artist_name'])
    return list(set(all_))

def albums_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['album_name'])
    return list(set(all_))

def tracks_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_uri'])
    return list(set(all_))

def track_names_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_name'])
    return list(set(all_))

def lyrics_in_playlist(playlist):
    all_ = {}
    info = {}
    ct = 0
    failed = []
    for track in playlist['tracks']:
        track_uri = track['track_uri']
        song_name = track['track_name']
        artist_name = track['artist_name']
        try:
            lyrics = lyricwikia.get_lyrics(artist_name, song_name)
            all_[track_uri] = lyrics
            info[track_uri] = (song_name,artist_name)
        except:
            failed.append(ct)
        ct += 1
    return all_,failed,info

f = open('../data/pooling/uri_to_track_info.pckl','rb')
uri_to_name_artist = pickle.load(f)
f.close()

f = open('../data/pooling/artist_to_songs.p','rb')
artist_to_songs = pickle.load(f)
f.close()

clean_artist_to_songs = {}
for artist in artist_to_songs.keys():
    clean_artist_to_songs[artist] = list(set(artist_to_songs[artist]))

name_artist_to_uri = {}
for uri in uri_to_name_artist.keys():
    tup = uri_to_name_artist[uri]
    name_artist_to_uri[tup] = uri

from gensim.models import Word2Vec
model = Word2Vec.load('../data/pooling/model_min5_new.bin')

In [5]:
def scrape_lyrics(pid):
    pl_turi,pl_auri = tracks.get_playlist(pid)
    lyrics_list = []
    ct = 0
    uris = []
    for uri in pl_turi:
        song,artist = uri_to_name_artist['spotify:track:'+uri]
        try:
            lyrics = lyricwikia.get_lyrics(artist, song)
            lyrics_list.append(lyrics.split())
            uris.append(uri)
        except:
            pass
    f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/scraped_lyrics_%s.p'%pid,'wb')
    pickle.dump(lyrics_list,f)
    f.close()
    f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/scraped_lyrics_uris_%s.p'%pid,'wb')
    pickle.dump(uris,f)
    f.close()
    return lyrics_list

In [6]:
# scrape_lyrics(1)

# pid = 1
# f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/scraped_lyrics_%s.p'%pid,'rb')
# lyrics_list = pickle.load(f)
# f.close()
# f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/scraped_lyrics_uris_%s.p'%pid,'rb')
# uris = pickle.load(f)
# f.close()

In [ ]:
# all_lyrics = []
for pid in range(1798,5000):
    if pid % 50 == 0:
        print('working on pid ',pid)
    lyrics_playlist = scrape_lyrics(pid)
#     all_lyrics += lyrics_playlist

working on pid  1800
working on pid  1850
working on pid  1900
working on pid  1950
working on pid  2000
working on pid  2050
working on pid  2100
working on pid  2150
working on pid  2200
working on pid  2250
working on pid  2300
working on pid  2350


In [ ]:
f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/all_lyrics.p','wb')
pickle.dump(all_lyrics,f)
f.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = [' '.join(words) for words in all_lyrics]
# create the transform
vectorizer = TfidfVectorizer(max_df = 0.9,min_df = 0.05)
# tokenize and build vocab
vectorizer.fit(text)
# # summarize
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# # encode document
# vector = vectorizer.transform([text[0]])
# # summarize encoded vector
# print(vector.shape)
# print(vector.toarray())

In [34]:
f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/all_lyrics.p','wb')
pickle.dump(text,f)
f.close()

In [ ]:
f = open('/Users/spandanmadan/Desktop/Spotify/Spotify/data/pooling/tfidf.p','wb')
pickle.dump(vectorizer,f)
f.close()